In [1]:
import pandas as pd
from collections import Counter
import numpy as np
import json
import re
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import json
import ast
from textblob_de import TextBlobDE as TextBlob

Dieses Notebook wird verwendet um die ganzen Cases und die Sätze nach Solr zu pushen 

## jura_docs_full

Einlesen der Datei

In [2]:
fileBasePath = "C:/Users/Sebastian/Desktop/Uni/NLP/NLPSTUDA/"

In [3]:
# Pfad zu csv MIT sentiments
df = pd.read_csv(fileBasePath+"23-12_cleaned_cases_with_sentiments.csv")

Indizes für die Iteration definieren

In [4]:
indices = [(None, 10000), (10000, 20000), (20000, 30000), (30000, 40000), (40000, 50000),(50000, 60000), (60000,70000), (70000,80000), (80000,90000), (90000,100000), (100000, None)]

In den cases sind court Informationen in einem Fehlerhaften json gespeichert, über diese Regex werrden diese Fehler gefunden, dann behoben und das json dann geparst

In [5]:
regExMakeJsonParsable = r": (.[^']*?)([,\}])"

Auch Geldbeträge müssen noch geparst werden. Wenn sie beispielsweise "Mrd." enthalten, wird das durch neun "0"er ersetzt.

In [6]:
def parseMoneyString(money):
    if ("Mrd. " in money):
        money = money.replace("Mrd. ", "")
        # add 9 0s to the string to replace the Mrd.
        money = money[:money.find(",")] + "0"*9 + money[money.find(","):]
    money = money.replace(".", "")
    money = money.replace("€", "")
    money = money.replace("\u20ac", "")
    money = money.replace(",", ".")
    return money

Diese Funktion führt ein Preprocessing durch und pusht dann ein data-Objekt in fullDocsDataObjects. Das Data-Objekt entspricht einem Solr-Document

In [7]:
counter = 0
def pushDataFull(row):
    global counter, fullDocsDataObjects
    core_name = "jura_docs_full"
    # defining the api-endpoint
    url = "http://localhost:8983/solr/"+core_name+"/update/json/docs"
    # data to be sent to api
    courtStr = row["court"]

    # not all values in the json contain a single quote "'" so lets add them
    subbedWithRegex = re.sub(regExMakeJsonParsable, ": '\g<1>'\g<2> ", courtStr)
    # however, python only accepts double quotes, so replace them
    subbed = subbedWithRegex.replace("\'", "\"")
    court = json.loads(subbed)
    # interpret the geldbetrag string as an array
    moneyAsArray = ast.literal_eval(row["geldbetrag"])
    # and parse it
    moneyAsArray = [parseMoneyString(m) for m in moneyAsArray]
    # create data object
    data = {
        "case_id":str(row["id"]),
        "content":str(row["content"]),
        "slug":str(row["slug"]),
        "court_id":str(court.get("id")),
        "court_name":str(court.get("name")),
        "court_slug":str(court.get("slug")),
        "court_city":str(court.get("city")),
        "court_state":str(court.get("state")),
        "court_jurisdiction":str(court.get("jurisdiction")),
        "court_level_of_appeal":str(court.get("level_of_appeal")),
        "file_number":str(row["file_number"]),
        "date":str(row["date"]),
        "created_date":str(row["created_date"]),
        "updated_date":str(row["updated_date"]),
        "type":str(row["type"]),
        "ecli":str(row["ecli"]),
        "sentiment":str(row["sentiment"]),
        "geldbetrag":moneyAsArray,
    }
    # push it to list
    fullDocsDataObjects.append(data)
    counter += 1
    if (counter % 1000 == 0):
        print("counter: ", counter)

Diese Funktion nimmt eine Liste an data-objekten (=Solr-documents) und pusht sie gleichzeitig an Solr. Anschließend wird comittet.

In [8]:
def pushToCore(core_name, jsonList):
    url = "http://localhost:8983/solr/"+core_name+"/update/json/docs"    
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    headers = {'Content-type': 'application/json'}

    #sending post request
    resp = session.post(url=url, data=json.dumps(jsonList), headers=headers)
    print(resp)
    # commit to solr
    requests.get("http://localhost:8983/solr/"+core_name+"/update?commit=true")

Mit den Indizes iterieren wir über die Daten. Die Funktion wird wieder zeilenweise auf den Ausschnitt (bestimmt durch die Indizes) des DataFrames angewendet.

In [9]:
counter = 0
for index in indices:
    fullDocsDataObjects = []
    df.iloc[index[0]:index[1]].apply(pushDataFull, axis=1)
    print("Prepared from ", index[0], " to ",index[1])
    print("Now pushing to solr")
    pushToCore(core_name="jura_docs_full", jsonList=fullDocsDataObjects)

counter:  1000
counter:  2000
counter:  3000
counter:  4000
counter:  5000
counter:  6000
counter:  7000
counter:  8000
counter:  9000
counter:  10000
Prepared from  None  to  10000
Now pushing to solr
<Response [200]>
counter:  11000
counter:  12000
counter:  13000
counter:  14000
counter:  15000
counter:  16000
counter:  17000
counter:  18000
counter:  19000
counter:  20000
Prepared from  10000  to  20000
Now pushing to solr
<Response [200]>
counter:  21000
counter:  22000
counter:  23000
counter:  24000
counter:  25000
counter:  26000
counter:  27000
counter:  28000
counter:  29000
counter:  30000
Prepared from  20000  to  30000
Now pushing to solr
<Response [200]>
counter:  31000
counter:  32000
counter:  33000
counter:  34000
counter:  35000
counter:  36000
counter:  37000
counter:  38000
counter:  39000
counter:  40000
Prepared from  30000  to  40000
Now pushing to solr
<Response [200]>
counter:  41000
counter:  42000
counter:  43000
counter:  44000
counter:  45000
counter:  4600

## jura_docs

Gleicher Ablauf für die einzelnen Sätze. Das data-Objekt enthält die Metadaten, die Infos zur Satzstruktur, den gesamten Satz, das mit TextBlobDe erzeugte Satz-Sentiment, das Case-Sentiment und den Geldbetrag.

In [9]:
counter = 0
def commitSolr():
    core_name = "jura_docs"
    requests.get("http://localhost:8983/solr/"+core_name+"/update?commit=true")

def prepareDataProcessed(caseId, sentenceNr, fullSentence, analysedSentence, metaDataRow):
    global counter, docsDataObjects

    courtStr = metaDataRow["court"]
    # not all values in the json contain a single quote "'" so lets add them
    subbedWithRegex = re.sub(regExMakeJsonParsable, ": '\g<1>'\g<2> ", courtStr)
    # however, python only accepts double quotes, so replace them
    subbed = subbedWithRegex.replace("\'", "\"")
    court = json.loads(subbed)
    moneyAsArray = ast.literal_eval(metaDataRow["geldbetrag"])
    moneyAsArray = [parseMoneyString(m) for m in moneyAsArray]
    blob = blob = TextBlob(fullSentence)

    data = {
    "case_id":str(case_id),
    "slug":str(metaDataRow["slug"]),
    "court_id":str(court.get("id")),
    "court_name":str(court.get("name")),
    "court_slug":str(court.get("slug")),
    "court_city":str(court.get("city")),
    "court_state":str(court.get("state")),
    "court_jurisdiction":str(court.get("jurisdiction")),
    "court_level_of_appeal":str(court.get("level_of_appeal")),
    "file_number":str(metaDataRow["file_number"]),
    "date":str(metaDataRow["date"]),
    "created_date":str(metaDataRow["created_date"]),
    "updated_date":str(metaDataRow["updated_date"]),
    "type":str(metaDataRow["type"]),
    "ecli":str(metaDataRow["ecli"]),
    "sentiment":str(metaDataRow["sentiment"]),
    "sentence_sentiment": float(blob.sentiment.polarity),
    "geldbetrag":moneyAsArray if len(moneyAsArray)>0 else [],
    "sentence_number": int(sentenceNr),
    "fullSentence": str(fullSentence),
    "root": analysedSentence["ROOT"] if analysedSentence["ROOT"] else [],
    "subjekt": analysedSentence["Subjekt"] if analysedSentence["Subjekt"] else [],
    "akkusativobjekt": analysedSentence["AkkusativObjekt"] if analysedSentence["AkkusativObjekt"] else [],
    "dativobjekt": analysedSentence["DativObjekt"] if analysedSentence["DativObjekt"] else [],
    "rest": analysedSentence["Rest"] if analysedSentence["Rest"] else []
    }
    docsDataObjects.append(data)

    if (counter % 1000 == 0):
        print("counter: ", counter)
    counter+=1

Iterieren über alle case_dicts, alle 10k Sätze Pushen

In [13]:
counter = 0
caseDictPath = "case_dict_"
for i in range(0,21):
    docsDataObjects = []
    # load json
    jsonFilePath = fileBasePath+caseDictPath+str(i)+"_1000.json"
    print("Current file: ", jsonFilePath)
    with open(jsonFilePath, "r", encoding="utf-8") as f:
        obj = json.load(f)
    print("File loaded. Now preprocessing.")
    for case_id in obj:
        # iterate over cases in json
        metaData = df[df.id == int(case_id)].iloc[0]
        for sentenceListIndex in obj[case_id]:
            # iterate over sentences in case
            try:
                # get the fullSentence
                fullSentence = obj[case_id][sentenceListIndex][0]
                # and its analysis from spacy
                analysedSentence = obj[case_id][sentenceListIndex][1]
                # push the dataobj to the array
                prepareDataProcessed(caseId = case_id,sentenceNr=sentenceListIndex, fullSentence=fullSentence, analysedSentence=analysedSentence,metaDataRow=metaData)
                if(counter == 10000):
                    print("Pushing 10k sentences")
                    pushToCore(core_name="jura_docs", jsonList=docsDataObjects)
                    del docsDataObjects
                    docsDataObjects = []
                    counter = 0
            except TypeError:
                pass
    print("Finised preprocessing of this file, final push")
    pushToCore(core_name="jura_docs", jsonList=docsDataObjects)
    del obj

nter:  6000
counter:  7000
counter:  8000
counter:  9000
Pushing 10k sentences
<Response [200]>
counter:  0
counter:  1000
counter:  2000
counter:  3000
counter:  4000
counter:  5000
counter:  6000
counter:  7000
counter:  8000
counter:  9000
Pushing 10k sentences
<Response [200]>
counter:  0
counter:  1000
counter:  2000
counter:  3000
counter:  4000
counter:  5000
counter:  6000
counter:  7000
counter:  8000
counter:  9000
Pushing 10k sentences
<Response [200]>
counter:  0
counter:  1000
counter:  2000
counter:  3000
counter:  4000
counter:  5000
counter:  6000
counter:  7000
counter:  8000
counter:  9000
Pushing 10k sentences
<Response [200]>
counter:  0
counter:  1000
counter:  2000
counter:  3000
counter:  4000
counter:  5000
counter:  6000
counter:  7000
counter:  8000
counter:  9000
Pushing 10k sentences
<Response [200]>
counter:  0
counter:  1000
counter:  2000
counter:  3000
counter:  4000
counter:  5000
counter:  6000
counter:  7000
counter:  8000
counter:  9000
Pushing 10k s

In [ ]:
del fullDocsDataObjects

counter = 0
for index in indices:
    # docsDataObjects = []
    # df.iloc[index[0]:index[1]].apply(pushDataFull, axis=1)
    # print("Prepared from ", index[0], " to ",index[1])
    # print("Now pushing to solr")
    pushToCore(core_name="jura_docs_full", jsonList=docsDataObjects[index[0]:index[1]])

In [21]:
counter, error_counter, counter-error_counter

(17044, 226, 16818)